# **Used Car Market Analysis & Pricing Intelligence**

In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Project_1/car_prices.csv')

In [ ]:
df.head(n=5)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


In [ ]:
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)

Missing Values:
 year                0
make            10301
model           10399
trim            10651
body            13195
transmission    65352
vin                 4
state               0
condition       11820
odometer           94
color             749
interior          749
seller              0
mmr                38
sellingprice       12
saledate           12
dtype: int64


**This dataset has missing values across multiple columns, but not all of them require the same treatment. Here’s how I preffered to prioritize and handle them effectively.**

1. VIN (Vehicle Identification Number):
Issue: The vin is a unique identifier for each vehicle.
Fix : vin is crucial for tracking unique cars, drop these rows.

In [ ]:
df.dropna(subset=["vin"], inplace=True)


2. Sale Date :
Issue: saledate is essential for time-based analysis.
Fix: The saledate column was converted to a standardized datetime64[ns] format using pd.to_datetime() to enable efficient time-series analysis and proper date hierarchies in Power BI. The errors='coerce' parameter ensures robustness by converting any invalid date strings to NaT for easier handling during preprocessing.

In [ ]:
# Convert 'saledate' to datetime format
df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')

# Check the conversion
print(df['saledate'].head())

# Optionally, save the cleaned dataset
df.to_csv('cleaned_dataset.csv', index=False)

print("Saledate has been converted to Date format and saved.")


<ipython-input-7-685b90d1ad7a>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
<ipython-input-7-685b90d1ad7a>:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')


0    2014-12-16 12:30:00+08:00
1    2014-12-16 12:30:00+08:00
2    2015-01-15 04:30:00+08:00
3    2015-01-29 04:30:00+08:00
4    2014-12-18 12:30:00+08:00
Name: saledate, dtype: object
Saledate has been converted to Date format and saved.


4. The condition column contained improperly formatted float values (e.g., 45.0 instead of 4.5) due to missing decimal points. We applied a transformation to detect and convert anomalous two-digit values greater than 5 into valid float scores by inserting a decimal between the digits, ensuring accurate numeric representation for downstream analysis.

In [ ]:
def fix_condition(val):
    try:
        val = float(val)

        # If value is greater than 5, assume it's like 45 → 4.5
        if val > 5 and val < 100:
            digits = str(int(val))
            if len(digits) == 2:
                return float(digits[0] + '.' + digits[1])

        # If it's already in the valid range, keep it
        if 0 <= val <= 5:
            return val

    except:
        return None

    return None


In [ ]:
df['condition_clean'] = df['condition'].apply(fix_condition)
print(df[['condition', 'condition_clean']].head(8))


   condition  condition_clean
0        5.0              5.0
1        5.0              5.0
2       45.0              4.5
3       41.0              4.1
4       43.0              4.3
5        1.0              1.0
6       34.0              3.4
7        2.0              2.0


5. Odometer : Issue : Values are in raw Miles format. Fix : The odometer column was converted from miles to kilometers using the standard factor (1 mile = 1.60934 km) to ensure consistency with the UK’s metric system. This transformation supports accurate distance-based analysis and aligns the dataset with international reporting standards.

In [ ]:
# Convert 'odometer' from miles to kilometers
df['odometer_km'] = df['odometer'] * 1.60934

# View the first 5 rows to confirm the conversion
print(df[['odometer', 'odometer_km']].head())

# Save the updated dataset
df.to_csv('cleaned_dataset.csv', index=False)

print("Odometer converted from miles to kilometers and saved.")

   odometer  odometer_km
0   16639.0  26777.80826
1    9393.0  15116.53062
2    1331.0   2142.03154
3   14282.0  22984.59388
4    2641.0   4250.26694
Odometer converted from miles to kilometers and saved.


Lets download this csv for file for further PowerBi Dashboard


In [ ]:
df.to_csv("cleaned_car_data.csv", index=False)


In [ ]:
from google.colab import files
files.download("cleaned_car_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>